#### Objective : 
The objective of this case study is to demonstrate how Artificial Neural Networks can be leveraged for classsification and how 
tuning an Artificial Neural Network can lead to an improvement in accuracy of predictions on either class.

#### Approach :
The dataset we have is infested with a high degreee of class imbalance. There are 39922 instaces that belong to classs 'no' while there are 5289 instances that belong to class 'yes'. Training the classification model on a dataset composed of samples drawn from such a dataset is likely to predict the classes belonging to the abundant class with greater correctness in comparision with oinstances that belong to the scarce class. We thus train the classification model on a dataset obtained by drawing an equal number of samples of either class and evaluating the performance of the model thus obtained on the remaining instances of the original dataset. In this way we can ensure that the performance of the model is balances on either classes i.e specificity and sensitivity values are almost identical.

#### About the Dataset :

Data Set Information :
DATA SOURCE : https://archive.ics.uci.edu/ml/machine-learning-databases/00222/

The dataset that we have has been derived from a marketing campaign run by a Portugese Banking Institution between 2008 and 2013. By training a classifier on the dataset we have, we want to evolve a model that can be used to asses the likelihood of a client subscribing to term deposit when contacted over the telephone. Clients which have a high likelihood of subscribing to the term deposit are accorded 1 and those having low likelihood are accorded 0 by the classifier.
________________________________________________________________________________________________________________________________

#### Input Features :
Bank Client data:

1 - age (numeric)

2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')

3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)

4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')

5 - default: has credit in default? (categorical: 'no','yes','unknown')

6 - housing: has housing loan? (categorical: 'no','yes','unknown')

7 - loan: has personal loan? (categorical: 'no','yes','unknown')

Related with the last contact of the current campaign :
8 - contact: contact communication type (categorical: 'cellular','telephone')

9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')

10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')

11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

Other attributes/input features :
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)

13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)

14 - previous: number of contacts performed before this campaign and for this client (numeric)

15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

Social and Economic Context attributes/input features :
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)

17 - cons.price.idx: consumer price index - monthly indicator (numeric)

18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)

19 - euribor3m: euribor 3 month rate - daily indicator (numeric)

20 - nr.employed: number of employees - quarterly indicator (numeric)

#### Output Feature / Target Feature :

21 - y - has the client subscribed a term deposit? (binary: 'yes','no')


#### 1) Importing the relevant Libraries :

In [1]:
import pandas as pd
import numpy as np

#### 2) Loading the dataset :

In [2]:
bank_data=pd.read_csv('bank-full.csv',sep=';')
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
age          45211 non-null int64
job          45211 non-null object
marital      45211 non-null object
education    45211 non-null object
default      45211 non-null object
balance      45211 non-null int64
housing      45211 non-null object
loan         45211 non-null object
contact      45211 non-null object
day          45211 non-null int64
month        45211 non-null object
duration     45211 non-null int64
campaign     45211 non-null int64
pdays        45211 non-null int64
previous     45211 non-null int64
poutcome     45211 non-null object
y            45211 non-null object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [3]:
bank_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
#checking for class imbalance:
bank_data['y'].value_counts()

no     39922
yes     5289
Name: y, dtype: int64

#### 3) Fetching the categorical attributes and their corresponding numerical indices from among the categorical input features "

In [5]:
#creating a dataframe of categorical input features
categorical_dataframe=bank_data.loc[:,'age':'poutcome'].select_dtypes(include=object)
#extracting the features/columns of the categorical dataframe
categorical_attributes=categorical_dataframe.columns.tolist()
#extracting the correspoding indices of categorical features
categorical_indices=[]
for attribute in categorical_attributes:
    categorical_indices.append(bank_data.columns.get_loc(attribute))
categorical_indices

[1, 2, 3, 4, 6, 7, 8, 10, 15]

#### 4) Label Encoding the categorical features :
Label Encoding is the process of assigning numerical labels to values contained within categorical input features of the dataframe. Label Encoding is performed inorder to facilitate the application of predictive mathematical models such as Logistic Regression, Support Vector Machines, Naive-Baye's etc, to those datasets which contain categorical/non-numerical data.

In [6]:
#label encoding the categorical attributes:
from sklearn.preprocessing import LabelEncoder
encoder_object=LabelEncoder()
for attribute in categorical_attributes:
    bank_data.loc[:,attribute]=encoder_object.fit_transform(bank_data.loc[:,attribute])
bank_data.head()

#label encoding the target feature:
bank_data.loc[:,'y']=encoder_object.fit_transform(bank_data.loc[:,'y'])

#### 5) OneHotEncoding the categorical features :
In order to facilitate the application of mathamatical models to datasets, merely assigning numerical labels to categorical attributes is simply not enough. One must remember that the assigned numerical labels are not related to each other in an ordinal sense, therefore we use a technique called 'OneHotEncoding' which, what basically does is, the following :

A column representing a categorical attribute is split into multiple columns such that we have new columns equal to the number of all the numerical labels used for encoding the values contained within the column under consideration. Inorder to expand upon what has just been stated, consider the following, the column of the dataframe named 'job' contains 41118 values, these 41118 values have been assigned numerical labels using integers from 0 to 12 i.e 13 integers. We will now split the 'job' column into 13 columns and each of the columns will represent an integer from 0 to 12.

For a particular observation (row index) if the job is encoded with a label '3', it will reflect in the newly created columns in the following way, the column that reprsents label '3' will be assigned 1 whereas rest of the columns will be assigned '0' and so on. This holds true for all the encoded categorical columns.

To sum up 'OneHotEncoding' can be described as the process of assigning a binary sequence of a particular 'length' to each value conatined within a 'LabelEncoded' attribute. The 'length'of the binary sequence is equal to the number of numerical labels used to represent the different values contained within a categorical column.

In [7]:
#onehotencoding:
from sklearn.preprocessing import OneHotEncoder
#while instantiating an object of the OneHotEncoder class we pass the indices of categorical features
encoder_object2=OneHotEncoder(categorical_features=categorical_indices)
bank_data=encoder_object2.fit_transform(bank_data).toarray()
bank_data=pd.DataFrame(data=bank_data)

#### 6) Checking the number of splits rendered to each categorical attribute/feature :

In [8]:
split_df=pd.DataFrame(data={'feature':categorical_attributes,'indices':categorical_indices,'splits':encoder_object2.n_values_})
split_df

,feature,indices,splits
0,job,1,12
1,marital,2,3
2,education,3,4
3,default,4,2
4,housing,6,2
5,loan,7,2
6,contact,8,3
7,month,10,12
8,poutcome,15,4


#### 7) Checking the transformed dataset:

In [9]:
bank_data.head()

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,58.0,2143.0,5.0,261.0,1.0,-1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,44.0,29.0,5.0,151.0,1.0,-1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,33.0,2.0,5.0,76.0,1.0,-1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,47.0,1506.0,5.0,92.0,1.0,-1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,33.0,1.0,5.0,198.0,1.0,-1.0,0.0,0.0


#### 8) Creating a training set containing equal number of samples of either class:

In [10]:
positive_data=bank_data[bank_data[51]==1].sample(n=3000,replace=False)
negative_data=bank_data[bank_data[51]==0].sample(n=3000,replace=False)
training_data=pd.concat([positive_data,negative_data])
training_data=training_data.reindex(np.random.permutation(training_data.index))

#removing the indices from the main dataset:
testing_data=bank_data.drop(training_data.index)

#### 9) Standardizing the training set and the test set:

In [11]:
from sklearn.preprocessing import StandardScaler
standardizer=StandardScaler()
training_data.iloc[:,0:51]=standardizer.fit_transform(training_data.iloc[:,0:51])
testing_data.iloc[:,0:51]=standardizer.transform(testing_data.iloc[:,0:51])

#### 9.2) Splitting the trainin data and testing data into sets of input features and target featur respectively:

In [12]:
X_train=training_data.iloc[:,0:51]
X_test=testing_data.iloc[:,0:51]
Y_train=training_data.iloc[:,51].values
Y_test=testing_data.iloc[:,51].values

In [13]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
32698,2.740554,-0.466404,-0.168683,-0.161763,-0.552191,-0.277848,-0.186175,-0.29655,-0.176366,-0.439679,...,-0.214109,-0.328684,0.578377,-0.950092,-0.467796,0.167986,1.779476,-0.565318,-0.476068,-0.379848
11367,-0.364890,-0.466404,-0.168683,-0.161763,1.810967,-0.277848,-0.186175,-0.29655,-0.176366,-0.439679,...,-0.214109,-0.328684,0.578377,1.071291,0.849745,0.406715,-0.837274,-0.565318,-0.476068,-0.379848
33364,-0.364890,-0.466404,-0.168683,-0.161763,-0.552191,-0.277848,5.371289,-0.29655,-0.176366,-0.439679,...,-0.214109,-0.328684,0.578377,-0.528971,-0.295404,0.526080,-0.380564,-0.565318,-0.476068,-0.379848
41549,-0.364890,-0.466404,-0.168683,-0.161763,-0.552191,-0.277848,-0.186175,-0.29655,-0.176366,2.274388,...,-0.214109,-0.328684,0.578377,0.060599,0.514087,-0.548202,-0.265668,-0.565318,-0.476068,-0.379848
31106,-0.364890,-0.466404,-0.168683,-0.161763,-0.552191,3.599095,-0.186175,-0.29655,-0.176366,-0.439679,...,-0.214109,-0.328684,0.578377,1.745086,-0.229576,0.167986,1.032654,-0.565318,-0.476068,-0.379848


#### 10) Creating the ANN:

1) From tensorflow.python.keras we import all the relevant libraries.

2) We define a function 'neural_network' which takes number of layers(layer_count), numbers of neurons(node_count) in each layer and the back propagation algorithm (opt) as its arguments and returns a neural network based classifier.

3) We are now supposed to tune the neural network based classifier by cross validating its performance across various combination of arguments fed to it.

4) Since the neural network is an object of the keras class we thus use a wrappers object to facilitate the tuning of the neural classifier through GridSearchCV. The wrappers object basically wraps the neural network in a form such that it can be adapted to the standard functionality of scikit-learn. 

5) We then employ an object of GridSeachCV and train it on the training data across various combinations of arguments/hyperparameters.

In [14]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
def neural_network(node_count,layer_count,opt):
    classifier=Sequential()
    input_layer=Dense(units=node_count,input_dim=51, activation='relu',kernel_initializer='uniform')
    classifier.add(input_layer)
    for count in range(layer_count):
        hidden_layer=Dense(units=node_count,activation='relu',kernel_initializer='uniform')
        classifier.add(hidden_layer)
    output_layer=Dense(units=1,activation='sigmoid',kernel_initializer='uniform')
    classifier.add(output_layer)
    classifier.compile(optimizer=opt,metrics=['accuracy'],loss='binary_crossentropy')
    return classifier

#importing the KerasClassifier inorder to facilitate hyperparameter tuning:
from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier
neural_classifier=KerasClassifier(build_fn=neural_network)
parameters={'node_count':[15,25,35,45],'layer_count':[2,3,4,5,6],'opt':['adam','rmsprop']}

#importing gridsearchcv:
from sklearn.model_selection import GridSearchCV
grid_object=GridSearchCV(estimator=neural_classifier,cv=5,scoring='accuracy',param_grid=parameters,verbose=4)
grid_object.fit(X_train,Y_train)

C:\User1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV] layer_count=2, node_count=15, opt=adam ..........................
Epoch 1/1
4800/4800 [==============================] - 1s 199us/step - loss: 6.2382 - acc: 0.5494
[CV]  layer_count=2, node_count=15, opt=adam, score=0.6033333333333334, total=   2.7s
[CV] layer_count=2, node_count=15, opt=adam ..........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.8s remaining:    0.0s


Epoch 1/1
4800/4800 [==============================] - 1s 263us/step - loss: 5.8499 - acc: 0.5704
[CV]  layer_count=2, node_count=15, opt=adam, score=0.6041666666666666, total=   3.0s
[CV] layer_count=2, node_count=15, opt=adam ..........................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.1s remaining:    0.0s


...

Epoch 1/1
4800/4800 [==============================] - 6s 1ms/step - loss: 7.7105 - acc: 0.5058
[CV]  layer_count=5, node_count=25, opt=adam, score=0.5175, total=  10.1s
[CV] layer_count=5, node_count=25, opt=rmsprop .......................
Epoch 1/1
4800/4800 [==============================] - 6s 1ms/step - loss: 7.5512 - acc: 0.5131
[CV]  layer_count=5, node_count=25, opt=rmsprop, score=0.5566666666666666, total=   9.9s
[CV] layer_count=5, node_count=25, opt=rmsprop .......................
Epoch 1/1
4800/4800 [==============================] - 6s 1ms/step - loss: 7.3061 - acc: 0.5281
[CV]  layer_count=5, node_count=25, opt=rmsprop, score=0.5341666666666667, total=   9.8s
[CV] layer_count=5, node_count=25, opt=rmsprop .......................
Epoch 1/1
4800/4800 [==============================] - 7s 1ms/step - loss: 7.2666 - acc: 0.5354
[CV]  layer_count=5, node_count=25, opt=rmsprop, score=0.4975, total=  10.0s
[CV] layer_count=5, node_count=25, opt=rmsprop .......................
Epo

Epoch 1/1
4800/4800 [==============================] - 9s 2ms/step - loss: 6.8683 - acc: 0.5537
[CV]  layer_count=6, node_count=15, opt=rmsprop, score=0.515, total=  13.7s
[CV] layer_count=6, node_count=15, opt=rmsprop .......................
Epoch 1/1
4800/4800 [==============================] - 9s 2ms/step - loss: 7.1214 - acc: 0.5373
[CV]  layer_count=6, node_count=15, opt=rmsprop, score=0.5383333333333333, total=  13.8s
[CV] layer_count=6, node_count=15, opt=rmsprop .......................
Epoch 1/1
4800/4800 [==============================] - 9s 2ms/step - loss: 7.1246 - acc: 0.5244
[CV]  layer_count=6, node_count=15, opt=rmsprop, score=0.5383333333333333, total=  13.9s
[CV] layer_count=6, node_count=25, opt=adam ..........................
Epoch 1/1
4800/4800 [==============================] - 10s 2ms/step - loss: 7.5805 - acc: 0.5096
[CV]  layer_count=6, node_count=25, opt=adam, score=0.5408333333333334, total=  15.1s
[CV] layer_count=6, node_count=25, opt=adam ..................

[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 34.6min finished


Epoch 1/1
6000/6000 [==============================] - 12s 2ms/step - loss: 6.0955 - acc: 0.5620


GridSearchCV(cv=5, error_score='raise',
       estimator=<tensorflow.python.keras._impl.keras.wrappers.scikit_learn.KerasClassifier object at 0x00000190E45F0860>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'node_count': [15, 25, 35, 45], 'layer_count': [2, 3, 4, 5, 6], 'opt': ['adam', 'rmsprop']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=4)

#### 11) Obtaining the best combination of arguments:

In [15]:
grid_object.best_params_

{'layer_count': 2, 'node_count': 15, 'opt': 'adam'}

#### 12) Using the obtained combination of arguments to obtain a neural network based classifier.

In [18]:
neural_classifier=neural_network(layer_count=2,node_count=15,opt='adam')
neural_classifier.fit(X_train,Y_train,epochs=100,batch_size=100)

Epoch 1/100
6000/6000 [==============================] - 12s 2ms/step - loss: 6.2780 - acc: 0.5608
Epoch 2/100
6000/6000 [==============================] - 0s 60us/step - loss: 5.4759 - acc: 0.5898
Epoch 3/100
6000/6000 [==============================] - 0s 61us/step - loss: 4.7302 - acc: 0.6132
Epoch 4/100
6000/6000 [==============================] - 0s 62us/step - loss: 4.0619 - acc: 0.6367
Epoch 5/100
6000/6000 [==============================] - 0s 63us/step - loss: 2.4396 - acc: 0.6283
Epoch 6/100
6000/6000 [==============================] - 0s 61us/step - loss: 0.8509 - acc: 0.6250
Epoch 7/100
6000/6000 [==============================] - 0s 61us/step - loss: 0.7013 - acc: 0.6617
Epoch 8/100
6000/6000 [==============================] - 0s 61us/step - loss: 0.6438 - acc: 0.6782
Epoch 9/100
6000/6000 [==============================] - 0s 67us/step - loss: 0.6024 - acc: 0.7013
Epoch 10/100
6000/6000 [==============================] - 0s 72us/step - loss: 0.5986 - acc: 0.7133
Epoch 11/

6000/6000 [==============================] - 0s 59us/step - loss: 0.3909 - acc: 0.8492
Epoch 83/100
6000/6000 [==============================] - 0s 59us/step - loss: 0.3799 - acc: 0.8497
Epoch 84/100
6000/6000 [==============================] - 0s 57us/step - loss: 0.3862 - acc: 0.8455
Epoch 85/100
6000/6000 [==============================] - 0s 56us/step - loss: 0.3800 - acc: 0.8487
Epoch 86/100
6000/6000 [==============================] - 0s 58us/step - loss: 0.3806 - acc: 0.8483
Epoch 87/100
6000/6000 [==============================] - 0s 58us/step - loss: 0.3796 - acc: 0.8478
Epoch 88/100
6000/6000 [==============================] - 0s 59us/step - loss: 0.3793 - acc: 0.8477
Epoch 89/100
6000/6000 [==============================] - 0s 60us/step - loss: 0.3752 - acc: 0.8512
Epoch 90/100
6000/6000 [==============================] - 0s 57us/step - loss: 0.3692 - acc: 0.8538
Epoch 91/100
6000/6000 [==============================] - 0s 57us/step - loss: 0.3790 - acc: 0.8482
Epoch 92/100


#### 13) Evaluating the performance of the classifier on the training set and the testing set.

In [26]:
#evaluating the performance on the training set:
Y_pred1=neural_classifier.predict(X_train)
from sklearn.preprocessing import binarize
Y_pred1=binarize(Y_pred1,threshold=0.50)
from sklearn.metrics import confusion_matrix,accuracy_score
print('PERFORMANCE ON TRAINING SET:')
cm1=confusion_matrix(Y_train,Y_pred1)
print(cm1)
print('Accuracy:',100*accuracy_score(Y_train,Y_pred1))
print('Specificity :',100*(cm1[0,0]/(cm1[0,0]+cm1[0,1])))
print('Sensitivity :',100*(cm1[1,1]/(cm1[1,0]+cm1[1,1])))

print('\n \n')

#evaluating the performance on the testing set:
Y_pred2=neural_classifier.predict(X_test)
Y_pred2=binarize(Y_pred2,threshold=0.50)
from sklearn.metrics import confusion_matrix,accuracy_score
cm2=confusion_matrix(Y_test,Y_pred2)
print('PERFORMANCE ON TESTING SET:')
print(cm2)
print('Accuracy:',100*accuracy_score(Y_test,Y_pred2))
print('Specificity :',100*(cm2[0,0]/(cm2[0,0]+cm2[0,1])))
print('Sensitivity :',100*(cm2[1,1]/(cm2[1,0]+cm[1,1])))



PERFORMANCE ON TRAINING SET:
[[2568  432]
 [ 400 2600]]
Accuracy: 86.13333333333333
Specificity : 85.6
Sensitivity : 86.66666666666667

 

PERFORMANCE ON TESTING SET:
[[30622  6300]
 [  358  1931]]
Accuracy: 83.02007089847237
Specificity : 82.9370023292346
Sensitivity : 84.35998252512013
